In [1]:
import os
import sys
import time
import glob
import numpy as np
import torch
import utils_imagenet
import logging
import argparse
import torch.nn as nn
import torch.utils
import torch.nn.functional as F
import torchvision.datasets as dset
import torch.backends.cudnn as cudnn

from torch.autograd import Variable
from model_search import Network
from architect import Architect





import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
import os
from PIL import Image
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
import numpy as np
from PIL import ImageFile
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import numpy as np
import pandas as pd
import os
import random 
from shutil import copyfile
from torch.utils.data import Dataset
from torchvision.datasets import ImageFolder
from PIL import Image
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import re
import albumentations as albu
from albumentations.pytorch import ToTensor
from catalyst.data import Augmentor
import torchxrayvision as xrv


import torch
import torchvision
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.utils.data import Dataset
import os
from PIL import Image
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
from PIL import Image
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
from sklearn.metrics import roc_auc_score
import re
import albumentations as albu
from albumentations.pytorch import ToTensor
from catalyst.data import Augmentor
from skimage.io import imread, imsave
import skimage




parser = argparse.ArgumentParser("cifar")
parser.add_argument('--data', type=str, default='../data', help='location of the data corpus')
parser.add_argument('--batch_size', type=int, default=64, help='batch size')
parser.add_argument('--learning_rate', type=float, default=0.025, help='init learning rate')
parser.add_argument('--learning_rate_min', type=float, default=0.001, help='min learning rate')
parser.add_argument('--momentum', type=float, default=0.9, help='momentum')
parser.add_argument('--weight_decay', type=float, default=3e-4, help='weight decay')
parser.add_argument('--report_freq', type=float, default=50, help='report frequency')
parser.add_argument('--gpu', type=int, default=0, help='gpu device id')
parser.add_argument('--epochs', type=int, default=50, help='num of training epochs')
parser.add_argument('--init_channels', type=int, default=16, help='num of init channels')
parser.add_argument('--layers', type=int, default=8, help='total number of layers')
parser.add_argument('--model_path', type=str, default='saved_models', help='path to save the model')
parser.add_argument('--cutout', action='store_true', default=False, help='use cutout')
parser.add_argument('--cutout_length', type=int, default=16, help='cutout length')
parser.add_argument('--drop_path_prob', type=float, default=0.3, help='drop path probability')
parser.add_argument('--save', type=str, default='EXP', help='experiment name')
parser.add_argument('--seed', type=int, default=2, help='random seed')
parser.add_argument('--grad_clip', type=float, default=5, help='gradient clipping')
parser.add_argument('--train_portion', type=float, default=0.5, help='portion of training data')
parser.add_argument('--unrolled', action='store_true', default=False, help='use one-step unrolled validation loss')
parser.add_argument('--arch_learning_rate', type=float, default=3e-4, help='learning rate for arch encoding')
parser.add_argument('--arch_weight_decay', type=float, default=1e-3, help='weight decay for arch encoding')
args = parser.parse_args([])



CIFAR_CLASSES = 10

/data/taoliu/anaconda3/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/data/taoliu/anaconda3/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/data/taoliu/anaconda3/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/data/taoliu/anaconda3/lib/python3.6/site-packages/tensorboar

In [2]:
np.random.seed(args.seed)
torch.cuda.set_device(args.gpu)
cudnn.benchmark = True
torch.manual_seed(args.seed)
cudnn.enabled=True
torch.cuda.manual_seed(args.seed)
logging.info('gpu device = %d' % args.gpu)
logging.info("args = %s", args)


In [3]:
criterion = nn.CrossEntropyLoss()
criterion = criterion.cuda()
model = Network(args.init_channels, CIFAR_CLASSES, args.layers, criterion)
model = model.cuda()
logging.info("param size = %fMB", utils_imagenet.count_parameters_in_MB(model))

/tmp/mozilla_taoliu0/COVID-CT-master/DenseNet169/utils_imagenet.py:83: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.



In [4]:
optimizer = torch.optim.SGD(
      model.parameters(),
      args.learning_rate,
      momentum=args.momentum,
      weight_decay=args.weight_decay)

train_transform, valid_transform = utils_imagenet._data_transforms_cifar10(args)
train_data = dset.CIFAR10(root=args.data, train=True, download=True, transform=train_transform)

num_train = len(train_data)
indices = list(range(num_train))
split = int(np.floor(num_train*0.01))

train_queue = torch.utils.data.DataLoader(
      train_data, batch_size=args.batch_size,
      sampler=torch.utils.data.sampler.SubsetRandomSampler(indices[:split]),
      pin_memory=True, num_workers=2)#######using all cifar10 data for supernet training


normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
train_transformer = transforms.Compose([
    transforms.Resize(48),
    transforms.RandomResizedCrop((32),scale=(0.5,1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize
])

val_transformer = transforms.Compose([
    transforms.Resize(32),
    transforms.CenterCrop(32),
    transforms.ToTensor(),
    normalize
])


# In[80]:


batchsize=4
def read_txt(txt_path):
    with open(txt_path) as f:
        lines = f.readlines()
    txt_data = [line.strip() for line in lines]
    return txt_data

class CovidCTDataset(Dataset):
    def __init__(self, root_dir, txt_COVID, txt_NonCOVID, transform=None):
        """
        Args:
            txt_path (string): Path to the txt file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        File structure:
        - root_dir
            - CT_COVID
                - img1.png
                - img2.png
                - ......
            - CT_NonCOVID
                - img1.png
                - img2.png
                - ......
        """
        self.root_dir = root_dir
        self.txt_path = [txt_COVID,txt_NonCOVID]
        self.classes = ['CT_COVID', 'CT_NonCOVID']
        self.num_cls = len(self.classes)
        self.img_list = []
        for c in range(self.num_cls):
            cls_list = [[os.path.join(self.root_dir,self.classes[c],item), c] for item in read_txt(self.txt_path[c])]
            self.img_list += cls_list
        self.transform = transform

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_path = self.img_list[idx][0]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)
        sample = {'img': image,
                  'label': int(self.img_list[idx][1])}
        return sample


Files already downloaded and verified


In [5]:
trainset = CovidCTDataset(root_dir='/tmp/mozilla_taoliu0/COVID-CT-master/Images-processed',
                          txt_COVID='/tmp/mozilla_taoliu0/COVID-CT-master/Data-split/COVID/trainCT_COVID.txt',
                          txt_NonCOVID='/tmp/mozilla_taoliu0/COVID-CT-master/Data-split/NonCOVID/trainCT_NonCOVID.txt',
                          transform= train_transformer)
valset = CovidCTDataset(root_dir='/tmp/mozilla_taoliu0/COVID-CT-master/Images-processed',
                          txt_COVID='/tmp/mozilla_taoliu0/COVID-CT-master/Data-split/COVID/valCT_COVID.txt',
                          txt_NonCOVID='/tmp/mozilla_taoliu0/COVID-CT-master/Data-split/NonCOVID/valCT_NonCOVID.txt',
                          transform= val_transformer)
testset = CovidCTDataset(root_dir='/tmp/mozilla_taoliu0/COVID-CT-master/Images-processed',
                          txt_COVID='/tmp/mozilla_taoliu0/COVID-CT-master/Data-split/COVID/testCT_COVID.txt',
                          txt_NonCOVID='/tmp/mozilla_taoliu0/COVID-CT-master/Data-split/NonCOVID/testCT_NonCOVID.txt',
                          transform= val_transformer)
print(trainset.__len__())
print(valset.__len__())
print(testset.__len__())

batchsize=4

train_loader = DataLoader(trainset, batch_size=batchsize, drop_last=True, shuffle=True,pin_memory=True, num_workers=2)
val_loader = DataLoader(valset, batch_size=batchsize, drop_last=True, shuffle=False,pin_memory=True, num_workers=2)
test_loader = DataLoader(testset, batch_size=batchsize, drop_last=True, shuffle=False,pin_memory=True, num_workers=2)

##we use the validation performance on COVID dataset after transfering on training dataset.


424
116
200


In [6]:
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, float(args.epochs), eta_min=args.learning_rate_min)


alpha = None
device = 'cuda'

model_covid = Network(args.init_channels, CIFAR_CLASSES, args.layers, criterion)
model_covid = model_covid.cuda()
model_covid.to(device)


def tranfer_weights(model,model_covid):
    model_covid = torch.nn.DataParallel(model_covid)   
    model = torch.nn.DataParallel(model)
    newmodel_dict=model_covid.state_dict()
    premodel_dict=model.state_dict()
    new_list=list(newmodel_dict.keys())
    pre_list=list(premodel_dict.keys())
    for i in range(4184):####The model contains 4186 keys, we need exclude the last layer
        newmodel_dict[new_list[i]]=premodel_dict[pre_list[i]]
    
    model_covid.load_state_dict(newmodel_dict)

    model_covid=model_covid.module
    return model_covid

model_covid=tranfer_weights(model,model_covid)


#dict_name=list(model.state_dict())
#for i,p in enumerate(dict_name):
    #print(i,p)

architect = Architect(model_covid, args)

In [17]:
optimizer_covid = optim.Adam(model_covid.parameters(), lr=0.0001)

def train_covid(train_loader, model_covid, criterion, optimizer):
    objs = utils_imagenet.AvgrageMeter()
    top1 = utils_imagenet.AvgrageMeter()
    model_covid.train()
    
    loss = 0
    train_correct = 0

    for step, batch_samples in enumerate(train_loader):
        
        input, target = batch_samples['img'].to(device), batch_samples['label'].to(device)
     
        target = target.cuda(async=True)
        input = input.cuda()
        input = Variable(input)
        target = Variable(target)

        optimizer.zero_grad()
        logits= model_covid(input)
        loss = criterion(logits, target)

        loss.backward()
        nn.utils.clip_grad_norm(model.parameters(), args.grad_clip)
        optimizer.step()




def train(train_queue, valid_queue, model,model_covid, architect, criterion, optimizer, lr):
    objs = utils_imagenet.AvgrageMeter()
    top1 = utils_imagenet.AvgrageMeter()
    top2 = utils_imagenet.AvgrageMeter()

    for step, (input, target) in enumerate(train_queue):
        model.train()
        n = input.size(0)

        input = Variable(input, requires_grad=False).cuda()
        target = Variable(target, requires_grad=False).cuda(async=True)

        # get a random minibatch from the search queue with replacement
        batch_next = next(iter(val_loader))
        input_search, target_search = batch_next['img'].to(device), batch_next['label'].to(device)

        input_search = Variable(input_search, requires_grad=False).cuda()
        target_search = Variable(target_search, requires_grad=False).cuda(async=True)
        
        train_covid(train_loader, model_covid, criterion, optimizer_covid)######train on covid

        architect.step(input, target, input_search, target_search, lr, optimizer, unrolled=False)

        optimizer.zero_grad()
        logits = model(input)
        loss = criterion(logits, target)

        loss.backward()
        nn.utils.clip_grad_norm(model.parameters(), args.grad_clip)
        optimizer.step()
        
        model_covid=tranfer_weights(model,model_covid)

        prec1, prec2 = utils_imagenet.accuracy(logits, target, topk=(1, 2))
        objs.update(loss.data, n)
        top1.update(prec1.data, n)
        top2.update(prec2.data, n)

        if step % args.report_freq == 0:
            logging.info('train %03d %e %f %f', step, objs.avg, top1.avg, top2.avg)

    return top1.avg, objs.avg




def infer(val_loader, model, criterion,epoch):
    
    objs = utils_imagenet.AvgrageMeter()
    top1 = utils_imagenet.AvgrageMeter()
    top5 = utils_imagenet.AvgrageMeter()
    
    model.eval()
    test_loss = 0
    correct = 0
    results = []
    
    TP = 0
    TN = 0
    FN = 0
    FP = 0
    
    with torch.no_grad():
        tpr_list = []
        fpr_list = []
        
        predlist=[]
        scorelist=[]
        targetlist=[]
        # Predict
        
    
        for batch_index, batch_samples in enumerate(val_loader):
            data, target = batch_samples['img'].to(device), batch_samples['label'].to(device)
            data = data[:, 0, :, :]
            data = data[:, None, :, :]
            data = data.repeat(1,3,1,1)
            data = Variable(data, volatile=True).cuda()
            target = Variable(target, volatile=True).cuda(async=True)            
            
            
            output = model(data)
            
            test_loss += criterion(output, target)
            score = F.softmax(output, dim=1)
            pred = output.argmax(dim=1, keepdim=True)
#             print('target',target.long()[:, 2].view_as(pred))
            correct += pred.eq(target.long().view_as(pred)).sum().item()
            
            targetcpu=target.long().cpu().numpy()
            predlist=np.append(predlist, pred.cpu().numpy())
            scorelist=np.append(scorelist, score.cpu().numpy()[:,1])
            targetlist=np.append(targetlist,targetcpu)
           
          
    return targetlist, scorelist, predlist






<input>:15: DeprecationWarning:

'async' and 'await' will become reserved keywords in Python 3.7

<ipython-input-17-e344b8e8ba3c>:15: DeprecationWarning:

'async' and 'await' will become reserved keywords in Python 3.7



In [14]:
for epoch in range(50):

    scheduler.step()
    lr = scheduler.get_lr()[0]
    logging.info('epoch %d lr %e', epoch, lr)

    genotype = model_covid.genotype()
    logging.info('genotype = %s', genotype)

    print(F.softmax(model_covid.alphas_normal, dim=-1))
    print(F.softmax(model_covid.alphas_reduce, dim=-1))

    # training
    train_acc, train_obj = train(train_queue, val_loader, model,model_covid, architect, criterion, optimizer, lr)
    
    targetlist, scorelist, predlist=infer(val_loader, model_covid, criterion,epoch)
    
    model_covid.genotype()

tensor([[0.1252, 0.1248, 0.1246, 0.1249, 0.1250, 0.1250, 0.1252, 0.1252],
        [0.1251, 0.1250, 0.1246, 0.1248, 0.1250, 0.1251, 0.1251, 0.1253],
        [0.1254, 0.1247, 0.1248, 0.1246, 0.1250, 0.1253, 0.1250, 0.1252],
        [0.1253, 0.1248, 0.1248, 0.1247, 0.1250, 0.1251, 0.1249, 0.1253],
        [0.1252, 0.1249, 0.1248, 0.1245, 0.1253, 0.1251, 0.1251, 0.1250],
        [0.1254, 0.1249, 0.1249, 0.1247, 0.1251, 0.1252, 0.1248, 0.1250],
        [0.1251, 0.1247, 0.1248, 0.1249, 0.1252, 0.1250, 0.1253, 0.1250],
        [0.1253, 0.1248, 0.1247, 0.1249, 0.1253, 0.1250, 0.1249, 0.1252],
        [0.1254, 0.1246, 0.1248, 0.1249, 0.1252, 0.1252, 0.1248, 0.1251],
        [0.1251, 0.1247, 0.1246, 0.1247, 0.1252, 0.1256, 0.1251, 0.1251],
        [0.1252, 0.1247, 0.1247, 0.1245, 0.1254, 0.1250, 0.1253, 0.1251],
        [0.1253, 0.1247, 0.1247, 0.1249, 0.1251, 0.1254, 0.1250, 0.1249],
        [0.1255, 0.1247, 0.1246, 0.1247, 0.1250, 0.1252, 0.1255, 0.1248],
        [0.1251, 0.1248, 0.1244, 0.124

/data/taoliu/anaconda3/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:472: DeprecationWarning:

To get the last learning rate computed by the scheduler, please use `get_last_lr()`.

/data/taoliu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: UserWarning:

torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.



tensor([[0.1255, 0.1245, 0.1242, 0.1245, 0.1252, 0.1253, 0.1255, 0.1253],
        [0.1251, 0.1250, 0.1245, 0.1248, 0.1248, 0.1250, 0.1254, 0.1254],
        [0.1257, 0.1245, 0.1245, 0.1243, 0.1249, 0.1256, 0.1253, 0.1251],
        [0.1252, 0.1249, 0.1248, 0.1248, 0.1249, 0.1249, 0.1251, 0.1253],
        [0.1254, 0.1249, 0.1248, 0.1245, 0.1251, 0.1249, 0.1252, 0.1252],
        [0.1256, 0.1245, 0.1245, 0.1243, 0.1254, 0.1254, 0.1250, 0.1253],
        [0.1253, 0.1246, 0.1246, 0.1248, 0.1254, 0.1250, 0.1253, 0.1249],
        [0.1255, 0.1247, 0.1245, 0.1246, 0.1255, 0.1252, 0.1248, 0.1254],
        [0.1256, 0.1244, 0.1246, 0.1246, 0.1255, 0.1249, 0.1249, 0.1254],
        [0.1253, 0.1245, 0.1244, 0.1244, 0.1251, 0.1259, 0.1251, 0.1253],
        [0.1254, 0.1244, 0.1244, 0.1243, 0.1255, 0.1251, 0.1255, 0.1254],
        [0.1255, 0.1245, 0.1245, 0.1247, 0.1252, 0.1255, 0.1249, 0.1251],
        [0.1257, 0.1244, 0.1243, 0.1245, 0.1251, 0.1255, 0.1256, 0.1250],
        [0.1254, 0.1245, 0.1242, 0.124

tensor([[0.1266, 0.1234, 0.1230, 0.1235, 0.1250, 0.1262, 0.1263, 0.1259],
        [0.1247, 0.1246, 0.1238, 0.1242, 0.1254, 0.1254, 0.1261, 0.1258],
        [0.1270, 0.1234, 0.1232, 0.1232, 0.1247, 0.1268, 0.1264, 0.1252],
        [0.1251, 0.1255, 0.1248, 0.1251, 0.1255, 0.1242, 0.1241, 0.1257],
        [0.1260, 0.1241, 0.1237, 0.1236, 0.1256, 0.1252, 0.1255, 0.1263],
        [0.1264, 0.1234, 0.1233, 0.1231, 0.1264, 0.1258, 0.1254, 0.1262],
        [0.1259, 0.1234, 0.1235, 0.1237, 0.1260, 0.1258, 0.1262, 0.1256],
        [0.1265, 0.1240, 0.1236, 0.1240, 0.1264, 0.1254, 0.1240, 0.1261],
        [0.1265, 0.1235, 0.1235, 0.1237, 0.1260, 0.1256, 0.1254, 0.1258],
        [0.1262, 0.1236, 0.1233, 0.1233, 0.1253, 0.1267, 0.1259, 0.1257],
        [0.1257, 0.1239, 0.1238, 0.1235, 0.1263, 0.1250, 0.1258, 0.1260],
        [0.1262, 0.1236, 0.1236, 0.1238, 0.1259, 0.1259, 0.1253, 0.1258],
        [0.1264, 0.1237, 0.1235, 0.1237, 0.1261, 0.1264, 0.1263, 0.1239],
        [0.1257, 0.1236, 0.1232, 0.123

tensor([[0.1279, 0.1221, 0.1218, 0.1223, 0.1244, 0.1276, 0.1273, 0.1265],
        [0.1241, 0.1252, 0.1235, 0.1244, 0.1248, 0.1251, 0.1258, 0.1271],
        [0.1281, 0.1216, 0.1215, 0.1216, 0.1260, 0.1280, 0.1274, 0.1258],
        [0.1244, 0.1263, 0.1253, 0.1259, 0.1261, 0.1231, 0.1234, 0.1254],
        [0.1270, 0.1226, 0.1221, 0.1222, 0.1261, 0.1257, 0.1271, 0.1272],
        [0.1274, 0.1219, 0.1219, 0.1217, 0.1273, 0.1266, 0.1264, 0.1268],
        [0.1263, 0.1227, 0.1226, 0.1229, 0.1272, 0.1252, 0.1272, 0.1259],
        [0.1278, 0.1228, 0.1225, 0.1230, 0.1282, 0.1252, 0.1240, 0.1266],
        [0.1274, 0.1224, 0.1224, 0.1228, 0.1267, 0.1259, 0.1262, 0.1263],
        [0.1270, 0.1225, 0.1222, 0.1222, 0.1256, 0.1278, 0.1267, 0.1259],
        [0.1256, 0.1237, 0.1232, 0.1230, 0.1278, 0.1239, 0.1259, 0.1269],
        [0.1268, 0.1224, 0.1224, 0.1227, 0.1270, 0.1259, 0.1261, 0.1267],
        [0.1272, 0.1229, 0.1226, 0.1229, 0.1278, 0.1269, 0.1269, 0.1228],
        [0.1262, 0.1226, 0.1222, 0.122

...

tensor([[0.1318, 0.1181, 0.1171, 0.1178, 0.1260, 0.1330, 0.1282, 0.1281],
        [0.1234, 0.1302, 0.1240, 0.1272, 0.1202, 0.1202, 0.1220, 0.1327],
        [0.1327, 0.1175, 0.1172, 0.1172, 0.1284, 0.1336, 0.1314, 0.1221],
        [0.1194, 0.1305, 0.1281, 0.1275, 0.1255, 0.1173, 0.1246, 0.1272],
        [0.1295, 0.1215, 0.1196, 0.1195, 0.1218, 0.1307, 0.1294, 0.1280],
        [0.1303, 0.1172, 0.1169, 0.1166, 0.1295, 0.1302, 0.1303, 0.1291],
        [0.1275, 0.1240, 0.1219, 0.1219, 0.1321, 0.1214, 0.1265, 0.1248],
        [0.1324, 0.1227, 0.1209, 0.1214, 0.1318, 0.1209, 0.1234, 0.1265],
        [0.1291, 0.1211, 0.1204, 0.1200, 0.1268, 0.1242, 0.1265, 0.1320],
        [0.1304, 0.1187, 0.1184, 0.1180, 0.1264, 0.1309, 0.1297, 0.1276],
        [0.1233, 0.1249, 0.1231, 0.1222, 0.1300, 0.1191, 0.1248, 0.1326],
        [0.1299, 0.1216, 0.1201, 0.1207, 0.1271, 0.1246, 0.1245, 0.1316],
        [0.1282, 0.1216, 0.1206, 0.1205, 0.1315, 0.1298, 0.1304, 0.1174],
        [0.1280, 0.1218, 0.1202, 0.120

tensor([[0.1323, 0.1176, 0.1163, 0.1170, 0.1266, 0.1336, 0.1282, 0.1283],
        [0.1234, 0.1313, 0.1242, 0.1276, 0.1193, 0.1197, 0.1211, 0.1335],
        [0.1335, 0.1171, 0.1166, 0.1165, 0.1286, 0.1345, 0.1320, 0.1213],
        [0.1189, 0.1311, 0.1283, 0.1275, 0.1250, 0.1163, 0.1249, 0.1279],
        [0.1300, 0.1216, 0.1193, 0.1189, 0.1215, 0.1314, 0.1293, 0.1281],
        [0.1305, 0.1169, 0.1164, 0.1161, 0.1295, 0.1307, 0.1302, 0.1297],
        [0.1277, 0.1245, 0.1219, 0.1219, 0.1326, 0.1215, 0.1256, 0.1242],
        [0.1331, 0.1234, 0.1212, 0.1217, 0.1319, 0.1199, 0.1232, 0.1256],
        [0.1290, 0.1217, 0.1206, 0.1200, 0.1262, 0.1240, 0.1257, 0.1328],
        [0.1311, 0.1182, 0.1178, 0.1174, 0.1265, 0.1308, 0.1300, 0.1281],
        [0.1226, 0.1256, 0.1234, 0.1223, 0.1300, 0.1184, 0.1244, 0.1332],
        [0.1301, 0.1219, 0.1199, 0.1205, 0.1268, 0.1242, 0.1240, 0.1326],
        [0.1281, 0.1219, 0.1205, 0.1203, 0.1317, 0.1304, 0.1306, 0.1166],
        [0.1282, 0.1221, 0.1201, 0.120

tensor([[0.1330, 0.1174, 0.1158, 0.1166, 0.1270, 0.1340, 0.1280, 0.1283],
        [0.1235, 0.1320, 0.1243, 0.1280, 0.1185, 0.1191, 0.1203, 0.1342],
        [0.1342, 0.1166, 0.1159, 0.1159, 0.1290, 0.1354, 0.1324, 0.1205],
        [0.1185, 0.1317, 0.1286, 0.1276, 0.1245, 0.1154, 0.1252, 0.1285],
        [0.1306, 0.1215, 0.1188, 0.1181, 0.1211, 0.1321, 0.1294, 0.1284],
        [0.1307, 0.1168, 0.1160, 0.1157, 0.1296, 0.1312, 0.1297, 0.1303],
        [0.1281, 0.1249, 0.1220, 0.1219, 0.1332, 0.1214, 0.1249, 0.1236],
        [0.1338, 0.1240, 0.1214, 0.1220, 0.1319, 0.1191, 0.1229, 0.1250],
        [0.1289, 0.1222, 0.1209, 0.1200, 0.1255, 0.1237, 0.1251, 0.1337],
        [0.1317, 0.1181, 0.1175, 0.1171, 0.1264, 0.1308, 0.1302, 0.1283],
        [0.1219, 0.1263, 0.1238, 0.1224, 0.1298, 0.1178, 0.1241, 0.1339],
        [0.1304, 0.1223, 0.1198, 0.1202, 0.1263, 0.1239, 0.1236, 0.1335],
        [0.1279, 0.1221, 0.1205, 0.1201, 0.1320, 0.1307, 0.1307, 0.1159],
        [0.1283, 0.1224, 0.1200, 0.119

tensor([[0.1336, 0.1171, 0.1152, 0.1161, 0.1277, 0.1343, 0.1278, 0.1281],
        [0.1236, 0.1326, 0.1244, 0.1283, 0.1180, 0.1186, 0.1196, 0.1348],
        [0.1350, 0.1162, 0.1153, 0.1153, 0.1294, 0.1363, 0.1328, 0.1197],
        [0.1180, 0.1322, 0.1291, 0.1278, 0.1241, 0.1145, 0.1255, 0.1288],
        [0.1312, 0.1214, 0.1183, 0.1175, 0.1206, 0.1328, 0.1294, 0.1289],
        [0.1310, 0.1168, 0.1156, 0.1154, 0.1298, 0.1316, 0.1292, 0.1306],
        [0.1283, 0.1254, 0.1222, 0.1220, 0.1337, 0.1214, 0.1241, 0.1229],
        [0.1343, 0.1248, 0.1217, 0.1224, 0.1321, 0.1183, 0.1222, 0.1241],
        [0.1287, 0.1229, 0.1213, 0.1201, 0.1248, 0.1234, 0.1243, 0.1345],
        [0.1323, 0.1179, 0.1170, 0.1166, 0.1263, 0.1309, 0.1304, 0.1287],
        [0.1212, 0.1270, 0.1243, 0.1228, 0.1295, 0.1170, 0.1237, 0.1345],
        [0.1307, 0.1225, 0.1196, 0.1200, 0.1261, 0.1234, 0.1233, 0.1343],
        [0.1277, 0.1223, 0.1205, 0.1199, 0.1325, 0.1309, 0.1308, 0.1154],
        [0.1285, 0.1225, 0.1199, 0.119

tensor([[0.1343, 0.1169, 0.1146, 0.1156, 0.1283, 0.1348, 0.1276, 0.1279],
        [0.1236, 0.1335, 0.1247, 0.1289, 0.1174, 0.1180, 0.1187, 0.1353],
        [0.1359, 0.1157, 0.1146, 0.1145, 0.1297, 0.1373, 0.1334, 0.1190],
        [0.1175, 0.1329, 0.1297, 0.1280, 0.1235, 0.1135, 0.1258, 0.1291],
        [0.1318, 0.1214, 0.1178, 0.1167, 0.1202, 0.1335, 0.1293, 0.1293],
        [0.1314, 0.1167, 0.1152, 0.1149, 0.1299, 0.1322, 0.1288, 0.1309],
        [0.1284, 0.1259, 0.1225, 0.1223, 0.1343, 0.1215, 0.1229, 0.1222],
        [0.1349, 0.1256, 0.1221, 0.1229, 0.1321, 0.1173, 0.1216, 0.1235],
        [0.1284, 0.1235, 0.1217, 0.1202, 0.1241, 0.1231, 0.1236, 0.1353],
        [0.1329, 0.1177, 0.1166, 0.1160, 0.1262, 0.1309, 0.1307, 0.1291],
        [0.1205, 0.1276, 0.1247, 0.1230, 0.1292, 0.1164, 0.1233, 0.1353],
        [0.1310, 0.1228, 0.1195, 0.1197, 0.1259, 0.1230, 0.1228, 0.1353],
        [0.1276, 0.1224, 0.1205, 0.1196, 0.1330, 0.1312, 0.1309, 0.1148],
        [0.1286, 0.1227, 0.1197, 0.119

/data/taoliu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:109: UserWarning:

volatile was removed and now has no effect. Use `with torch.no_grad():` instead.

